[![Open In Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/github/QuantLet/EMQA/blob/main/EMQA_volatility_clustering/EMQA_volatility_clustering.ipynb)

# EMQA_volatility_clustering

Volatility clustering in oil returns.

**Output:** `lecture1_volatility_clustering.pdf`

In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import warnings
warnings.filterwarnings('ignore')

plt.rcParams.update({
    'figure.facecolor': 'none',
    'axes.facecolor': 'none',
    'savefig.facecolor': 'none',
    'savefig.transparent': True,
    'axes.grid': False,
    'axes.spines.top': False,
    'axes.spines.right': False,
    'font.size': 11,
    'figure.figsize': (12, 6),
})

COLORS = {
    'blue': '#1A3A6E', 'red': '#CD0000', 'green': '#2E7D32',
    'orange': '#E67E22', 'purple': '#8E44AD', 'gray': '#808080',
    'cyan': '#00BCD4', 'amber': '#B5853F'
}

def save_fig(fig, name):
    fig.savefig(name, bbox_inches='tight', transparent=True, dpi=300)
    print(f"Saved: {name}")


In [ ]:
import yfinance as yf

def fetch(ticker, start='2020-01-01', end='2025-12-31'):
    d = yf.download(ticker, start=start, end=end, progress=False)
    if isinstance(d.columns, pd.MultiIndex):
        return d['Close'].squeeze().dropna()
    return d['Close'].dropna()


In [ ]:
brent = fetch('BZ=F')
ret = np.log(brent/brent.shift(1)).dropna()
rvol = ret.rolling(30).std() * np.sqrt(252) * 100

fig, (ax1, ax2) = plt.subplots(2, 1, figsize=(14, 8), sharex=True)
ax1.plot(ret.index, ret*100, color=COLORS['blue'], lw=0.4, alpha=0.8)
ax1.axhline(0, color='gray', lw=0.5)
for start, end, label in [('2020-02', '2020-06', 'COVID'), ('2022-02', '2022-06', 'Ukraine')]:
    ax1.axvspan(pd.Timestamp(start), pd.Timestamp(end), alpha=0.15, color=COLORS['red'])
    ax1.text(pd.Timestamp(start), ax1.get_ylim()[1]*0.9, label, fontsize=8, color=COLORS['red'])
ax1.set_title('Oil Returns: "Big moves follow big moves"', fontweight='bold')
ax1.set_ylabel('Daily Return (%)')

ax2.plot(rvol.index, rvol, color=COLORS['red'], lw=1)
ax2.fill_between(rvol.index, 0, rvol, alpha=0.15, color=COLORS['red'])
ax2.set_title('30-Day Rolling Annualized Volatility', fontweight='bold')
ax2.set_ylabel('Volatility (%)')

fig.legend(['Returns', 'Volatility'], loc='upper center', bbox_to_anchor=(0.5, -0.01), ncol=2, frameon=False)
fig.tight_layout()
save_fig(fig, 'lecture1_volatility_clustering.pdf')
plt.show()
